# At Home Solar Charging notebooks: Exploring how EV charging aligns with rooftop solar generation by homes

<p>We will be using Pecan Street Inc. data from Dataport to determine how electric vehicle charging aligns with rooftop solar generation.
    
<br>Data from 24 homes with fairly complete data for the year 2018 is used to explore this question.
    
<br>
Pecans Streets data can be obtained by applying for a dataport account at https://www.dataport.pecanstreet.org.</p>


In [ ]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
import sys
sys.path.insert(0,'..')
from config.read_config import get_database_config
import numpy as np
%matplotlib inline
sys.executable  # shows you your path to the python you're using

## We've already extracted the data for you into the 4 seasons of data for the time period and made them available on the JupyterHub server in the directory `/shared/JupyterHub-Examples-Data/ev/`

If you wish to manipulate the queries for your own or work with the data from a live database connection, see the notebook named `Data-Extraction--EV_charging_alignment_with_at_home_rooftop_solar.ipynb` in this same directory.
That notebook goes through the initial home selection and data extraction, then exports those files to a zipped csv file.

In [ ]:
# read fall data from the prepared csv.zip file
fall_df = pd.read_csv('/shared/JupyterHub-Examples-Data/ev/ev_charging_alignmnent_fall.zip', compression='zip')

fall_df.head(15)

In [ ]:
# read spring data from the prepared csv.zip file
spring_df = pd.read_csv('/shared/JupyterHub-Examples-Data/ev/ev_charging_alignmnent_spring.zip', compression='zip')

spring_df.head(15)

In [ ]:
# read fall data from the prepared csv.zip file
summer_df = pd.read_csv('/shared/JupyterHub-Examples-Data/ev/ev_charging_alignmnent_summer.zip', compression='zip')

summer_df.head(15)

In [ ]:
# read winter data from the prepared csv.zip file
winter_df = pd.read_csv('/shared/JupyterHub-Examples-Data/ev/ev_charging_alignmnent_winter.zip', compression='zip')

winter_df.head(15)

In [ ]:
# data analysis for summer

#replace null with 0's
summer_df = summer_df.fillna(0)
# convert localminute to pandas daytime type
summer_df['datetime'] = pd.to_datetime(summer_df['localminute'])

summer_df = summer_df.set_index('datetime')

#create hour column
summer_df['hr'] = summer_df.index.hour

#create new dataframes with solar, car1, and hour column
summer_df_new = pd.DataFrame(summer_df, columns = ['solar', 'car1', 'hr'])

# group data based on hour and get avg
summer_df_grouped = summer_df_new.groupby(['hr']).mean()
plot_summer = summer_df_grouped.reset_index()

In [ ]:
# plot summer
plt.figure(figsize=(20,10))
plt.plot(plot_summer['hr'], plot_summer['car1'], label="car1")
plt.plot(plot_summer['hr'], plot_summer['solar'], label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
# Total car1 usuage powered by solar in summer
total_car1_summer = summer_df_new['car1'].sum()
solar_car1_summer = summer_df_new.loc[summer_df_new['car1'] < summer_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_summer = (solar_car1_summer/total_car1_summer) * 100
car1_powered_by_solar_summer

In [ ]:
# data analysis for fall

#replace null with 0's
fall_df = fall_df.fillna(0)
# convert localminute to pandas daytime type
fall_df['datetime'] = pd.to_datetime(fall_df['localminute'])

fall_df = fall_df.set_index('datetime')

#create hour column
fall_df['hr'] = fall_df.index.hour

#create new dataframes with solar, car1, and hour column
fall_df_new = pd.DataFrame(fall_df, columns = ['solar', 'car1', 'hr'])

# group data based on hour and get avg
fall_df_grouped = fall_df_new.groupby(['hr']).mean()
plot_fall = fall_df_grouped.reset_index()

In [ ]:
# plot fall
plt.figure(figsize=(20,10))
plt.plot(plot_fall['hr'], plot_fall['car1'], label="car1")
plt.plot(plot_fall['hr'], plot_fall['solar'], label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
# Total car1 usuage powered by solar in fall
total_car1_fall = fall_df_new['car1'].sum()
solar_car1_fall = fall_df_new.loc[fall_df_new['car1'] < fall_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_fall = (solar_car1_fall/total_car1_fall) * 100
car1_powered_by_solar_fall

In [ ]:
#data analysis for spring

#replace null with 0's 
spring_df = spring_df.fillna(0)


# convert localminute to pandas datetime type
spring_df['datetime'] = pd.to_datetime(spring_df['localminute'])

spring_df = spring_df.set_index('datetime')

#create hour column. We will be calculating average hourly load for spring.
spring_df['hr'] = spring_df.index.hour

#create new dataframes with only solar, car1 and hour column
spring_df_new = pd.DataFrame(spring_df, columns = ['solar', 'car1','hr'])

#group data based on hour and take avg
spring_df_grouped = spring_df_new.groupby(['hr']).mean()
plot_spring = spring_df_grouped.reset_index()

In [ ]:
#plot spring
plt.figure(figsize=(20,10))
plt.plot(plot_spring['hr'],plot_spring['car1'],label="car1")
plt.plot(plot_spring['hr'],plot_spring['solar'],label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
#Total car1 usage powered by PV system in spring
total_car1_spring = spring_df_new['car1'].sum()
solar_car1_spring = spring_df_new.loc[spring_df_new['car1'] < spring_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_spring = (solar_car1_spring/total_car1_spring)*100
car1_powered_by_solar_spring

In [ ]:
#data analysis for winter

#replace null with 0's 
winter_df = winter_df.fillna(0)

# convert localminute to pandas datetime type
winter_df['datetime'] = pd.to_datetime(winter_df['localminute'])

winter_df = winter_df.set_index('datetime')

#create hour column. We will be calculating average hourly load for winter.
winter_df['hr'] = winter_df.index.hour

#create new dataframes with only solar, car1 and hour column
winter_df_new = pd.DataFrame(winter_df, columns = ['solar', 'car1','hr'])

#group data based on hour and take avg
winter_df_grouped = winter_df_new.groupby(['hr']).mean()
plot_winter = winter_df_grouped.reset_index()

In [ ]:
#plot winter
plt.figure(figsize=(20,10))
plt.plot(plot_winter['hr'],plot_winter['car1'],label="car1")
plt.plot(plot_winter['hr'],plot_winter['solar'],label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
#Total car1 usage powered by PV system in winter
total_car1_winter = winter_df_new['car1'].sum()
solar_car1_winter = winter_df_new.loc[winter_df_new['car1'] < winter_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_winter = (solar_car1_winter/total_car1_winter) * 100
car1_powered_by_solar_winter

In [ ]:
import statistics
statistics.mean([car1_powered_by_solar_winter, car1_powered_by_solar_spring, car1_powered_by_solar_summer, car1_powered_by_solar_fall])

## Observations:
From the above graphs through out the year most homes started EV charging arount 9PM to 9:30PM and ended charging around 6AM (outside of solar peak hours). The average percentage of solar powered used for EV charging is around 4.4 %